# In this file, we will import several models of specific cyber attack classifier (based on DecisionTreeClassifier, best classfier)
### We need to import (else we will create new models)
- Classification
- All specific attack models

#### Libraries importation

In [1]:
# Libraries
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, PassiveAggressiveClassifier, Perceptron, RidgeClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import joblib
import gc

# Path to datasets
DATASET_DIRECTORY = ".\Files\\"

# Classification training (if you already have classification model in your Models repository, you can go to the next section)
### Table of content
- Importing DataSet
- Scaling
- Classification

#### Importing DataSet

In [2]:
# Get all the datasets in the directory and sort them
df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()

# Only use a part of all datasets (100 files)
# df_sets = df_sets[0:10]

# Set 80% of the datasets as training sets and 20% as test sets
training_sets = df_sets[:int(len(df_sets)*.8)]
test_sets = df_sets[int(len(df_sets)*.8):]

# Define each column of the dataset and the target column
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count',
       'syn_count', 'fin_count', 'urg_count', 'rst_count', 
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
       'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
       'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight', 
]
y_column = 'label'

  0%|          | 0/135 [00:00<?, ?it/s]

100%|██████████| 135/135 [01:37<00:00,  1.38it/s]


#### Scaling and Classification

In [6]:
# Turn variable to True if you want to save the model
save_model = False

# Turn variable to True if you want to load the model
load_model = True

# Try to load the model
try:
    ML_model = joblib.load(".\Models\model_DecisionTreeClassifier_34_classes.joblib")
except FileNotFoundError:
    load_model = False
    save_model = True
    print("No Model found in Models directory named 'model_DecisionTreeClassifier_34_classes.joblib'. DecisionTreeClassifier model creation beginning...")
    exit()    

# Define the name of the model
ML_neam = "DecisionTreeClassifier"

# If Model found in Models directory named 'model_DecisionTreeClassifier_34_classes.joblib'
if not load_model:
    # Define the scaler method
    scaler = StandardScaler()

    # For each training set
    for train_set in tqdm(training_sets):
        # Fit the scaler on the training sets
        scaler.fit(pd.read_csv(DATASET_DIRECTORY + train_set)[X_columns])

    # Definition of the models to use and their names
    ML_model = DecisionTreeClassifier()

    # For each dataset of the training set
    for train_set in tqdm(training_sets):
        # Load the dataset
        d = pd.read_csv(DATASET_DIRECTORY + train_set)

        # Normalize the dataset
        d[X_columns] = scaler.transform(d[X_columns])

        # Train the model
        ML_model.fit(d[X_columns], d[y_column])

        # Delete the dataset from the memory
        del d  

# Initialize the list of true labels
y_test = []

# Initialize the list of predictions
y_pred = []

# Initialize the list of test sets
d_test = []

# For each dataset of the test set
for test_set in tqdm(test_sets):
    # Load the dataset
    d_test_current = pd.read_csv(DATASET_DIRECTORY + test_set)

    # Normalize the dataset
    d_test_current[X_columns] = scaler.transform(d_test_current[X_columns])

    # Add the true labels to the list
    y_test += list(d_test_current[y_column].values)

    # Predict the labels
    y_pred += list(ML_model.predict(d_test_current[X_columns]))

    d_test += list(d_test_current[X_columns].values)

    # print("y_test : ",len(y_test), "y_pred : ",len(y_pred), "d_test : ",len(d_test))

    # Delete the dataset from the memory
    del d_test_current

# For each prediction
print("Prediction results:")
print()

# If the user wants to save the models
if save_model:
    # Save the model
    joblib.dump(ML_model, f".\Models\model_{ML_neam}_34_classes.joblib")

#print('y_pred: ', len(y_pred))
#print('y_test: ', len(y_test))
print(f"##### {ML_neam} (34 classes) #####")
print('accuracy_score: ', accuracy_score(y_pred, y_test))
print('recall_score: ', recall_score(y_pred, y_test, average='macro'))
print('precision_score: ', precision_score(y_pred, y_test, average='macro'))
print('f1_score: ', f1_score(y_pred, y_test, average='macro'))
print()

# Flush the memory
# gc.collect()

  0%|          | 0/34 [00:00<?, ?it/s]

100%|██████████| 34/34 [00:53<00:00,  1.56s/it]


Prediction results:

##### DecisionTreeClassifier (34 classes) #####
accuracy_score:  0.9919546707251463
recall_score:  0.8066593236051665
precision_score:  0.8081786041737559
f1_score:  0.8058064378537952



#### Dictionnary for 7 classes

In [7]:
dict_7classes = {}
dict_7classes['DDoS-RSTFINFlood'] = 'DDoS'
dict_7classes['DDoS-PSHACK_Flood'] = 'DDoS'
dict_7classes['DDoS-SYN_Flood'] = 'DDoS'
dict_7classes['DDoS-UDP_Flood'] = 'DDoS'
dict_7classes['DDoS-TCP_Flood'] = 'DDoS'
dict_7classes['DDoS-ICMP_Flood'] = 'DDoS'
dict_7classes['DDoS-SynonymousIP_Flood'] = 'DDoS'
dict_7classes['DDoS-ACK_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-UDP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-ICMP_Fragmentation'] = 'DDoS'
dict_7classes['DDoS-SlowLoris'] = 'DDoS'
dict_7classes['DDoS-HTTP_Flood'] = 'DDoS'

dict_7classes['DoS-UDP_Flood'] = 'DoS'
dict_7classes['DoS-SYN_Flood'] = 'DoS'
dict_7classes['DoS-TCP_Flood'] = 'DoS'
dict_7classes['DoS-HTTP_Flood'] = 'DoS'


dict_7classes['Mirai-greeth_flood'] = 'Mirai'
dict_7classes['Mirai-greip_flood'] = 'Mirai'
dict_7classes['Mirai-udpplain'] = 'Mirai'

dict_7classes['Recon-PingSweep'] = 'Recon'
dict_7classes['Recon-OSScan'] = 'Recon'
dict_7classes['Recon-PortScan'] = 'Recon'
dict_7classes['VulnerabilityScan'] = 'Recon'
dict_7classes['Recon-HostDiscovery'] = 'Recon'

dict_7classes['DNS_Spoofing'] = 'Spoofing'
dict_7classes['MITM-ArpSpoofing'] = 'Spoofing'

dict_7classes['BenignTraffic'] = 'Benign'

dict_7classes['BrowserHijacking'] = 'Web'
dict_7classes['Backdoor_Malware'] = 'Web'
dict_7classes['XSS'] = 'Web'
dict_7classes['Uploading_Attack'] = 'Web'
dict_7classes['SqlInjection'] = 'Web'
dict_7classes['CommandInjection'] = 'Web'


dict_7classes['DictionaryBruteForce'] = 'BruteForce'

#### Dictionaries for specific models

In [8]:
# Define the 2 classes for each attack type to create a specific model for each attack type
# DDoS :
dict_2classes_DDoS = {'DDoS-RSTFINFlood': 'DDoS', 'DDoS-PSHACK_Flood': 'DDoS', 'DDoS-SYN_Flood': 'DDoS', 'DDoS-UDP_Flood': 'DDoS', 'DDoS-TCP_Flood': 'DDoS', 'DDoS-ICMP_Flood': 'DDoS', 'DDoS-SynonymousIP_Flood': 'DDoS', 'DDoS-ACK_Fragmentation': 'DDoS', 'DDoS-UDP_Fragmentation': 'DDoS', 'DDoS-ICMP_Fragmentation': 'DDoS', 'DDoS-SlowLoris': 'DDoS', 'DDoS-HTTP_Flood': 'DDoS',
                      'DoS-UDP_Flood': 'Non-DDoS', 'DoS-SYN_Flood': 'Non-DDoS', 'DoS-TCP_Flood': 'Non-DDoS', 'DoS-HTTP_Flood': 'Non-DDoS', 
                      'Mirai-greeth_flood': 'Non-DDoS', 'Mirai-greip_flood': 'Non-DDoS', 'Mirai-udpplain': 'Non-DDoS', 
                      'Recon-PingSweep': 'Non-DDoS', 'Recon-OSScan': 'Non-DDoS', 'Recon-PortScan': 'Non-DDoS', 'VulnerabilityScan': 'Non-DDoS', 'Recon-HostDiscovery': 'Non-DDoS', 
                      'DNS_Spoofing': 'Non-DDoS', 'MITM-ArpSpoofing': 'Non-DDoS', 
                      'BenignTraffic': 'Non-DDoS', 
                      'BrowserHijacking': 'Non-DDoS', 'Backdoor_Malware': 'Non-DDoS', 'XSS': 'Non-DDoS', 'Uploading_Attack': 'Non-DDoS', 'SqlInjection': 'Non-DDoS', 'CommandInjection': 'Non-DDoS', 
                      'DictionaryBruteForce': 'Non-DDoS'}

# DoS :
dict_2classes_DoS = {'DDoS-RSTFINFlood': 'Non-DoS', 'DDoS-PSHACK_Flood': 'Non-DoS', 'DDoS-SYN_Flood': 'Non-DoS', 'DDoS-UDP_Flood': 'Non-DoS', 'DDoS-TCP_Flood': 'Non-DoS', 'DDoS-ICMP_Flood': 'Non-DoS', 'DDoS-SynonymousIP_Flood': 'Non-DoS', 'DDoS-ACK_Fragmentation': 'Non-DoS', 'DDoS-UDP_Fragmentation': 'Non-DoS', 'DDoS-ICMP_Fragmentation': 'Non-DoS', 'DDoS-SlowLoris': 'Non-DoS', 'DDoS-HTTP_Flood': 'Non-DoS',
                      'DoS-UDP_Flood': 'DoS', 'DoS-SYN_Flood': 'DoS', 'DoS-TCP_Flood': 'DoS', 'DoS-HTTP_Flood': 'DoS', 
                      'Mirai-greeth_flood': 'Non-DoS', 'Mirai-greip_flood': 'Non-DoS', 'Mirai-udpplain': 'Non-DoS', 
                      'Recon-PingSweep': 'Non-DoS', 'Recon-OSScan': 'Non-DoS', 'Recon-PortScan': 'Non-DoS', 'VulnerabilityScan': 'Non-DoS', 'Recon-HostDiscovery': 'Non-DoS', 
                      'DNS_Spoofing': 'Non-DoS', 'MITM-ArpSpoofing': 'Non-DoS', 
                      'BenignTraffic': 'Non-DoS', 
                      'BrowserHijacking': 'Non-DoS', 'Backdoor_Malware': 'Non-DoS', 'XSS': 'Non-DoS', 'Uploading_Attack': 'Non-DoS', 'SqlInjection': 'Non-DoS', 'CommandInjection': 'Non-DoS', 
                      'DictionaryBruteForce': 'Non-DoS'}

# Mirai :
dict_2classes_Mirai = {'DDoS-RSTFINFlood': 'Non-Mirai', 'DDoS-PSHACK_Flood': 'Non-Mirai', 'DDoS-SYN_Flood': 'Non-Mirai', 'DDoS-UDP_Flood': 'Non-Mirai', 'DDoS-TCP_Flood': 'Non-Mirai', 'DDoS-ICMP_Flood': 'Non-Mirai', 'DDoS-SynonymousIP_Flood': 'Non-Mirai', 'DDoS-ACK_Fragmentation': 'Non-Mirai', 'DDoS-UDP_Fragmentation': 'Non-Mirai', 'DDoS-ICMP_Fragmentation': 'Non-Mirai', 'DDoS-SlowLoris': 'Non-Mirai', 'DDoS-HTTP_Flood': 'Non-Mirai',
                      'DoS-UDP_Flood': 'Non-Mirai', 'DoS-SYN_Flood': 'Non-Mirai', 'DoS-TCP_Flood': 'Non-Mirai', 'DoS-HTTP_Flood': 'Non-Mirai', 
                      'Mirai-greeth_flood': 'Mirai', 'Mirai-greip_flood': 'Mirai', 'Mirai-udpplain': 'Mirai', 
                      'Recon-PingSweep': 'Non-Mirai', 'Recon-OSScan': 'Non-Mirai', 'Recon-PortScan': 'Non-Mirai', 'VulnerabilityScan': 'Non-Mirai', 'Recon-HostDiscovery': 'Non-Mirai', 
                      'DNS_Spoofing': 'Non-Mirai', 'MITM-ArpSpoofing': 'Non-Mirai', 
                      'BenignTraffic': 'Non-Mirai', 
                      'BrowserHijacking': 'Non-Mirai', 'Backdoor_Malware': 'Non-Mirai', 'XSS': 'Non-Mirai', 'Uploading_Attack': 'Non-Mirai', 'SqlInjection': 'Non-Mirai', 'CommandInjection': 'Non-Mirai', 
                      'DictionaryBruteForce': 'Non-Mirai'}

# Recon :
dict_2classes_Recon = {'DDoS-RSTFINFlood': 'Non-Recon', 'DDoS-PSHACK_Flood': 'Non-Recon', 'DDoS-SYN_Flood': 'Non-Recon', 'DDoS-UDP_Flood': 'Non-Recon', 'DDoS-TCP_Flood': 'Non-Recon', 'DDoS-ICMP_Flood': 'Non-Recon', 'DDoS-SynonymousIP_Flood': 'Non-Recon', 'DDoS-ACK_Fragmentation': 'Non-Recon', 'DDoS-UDP_Fragmentation': 'Non-Recon', 'DDoS-ICMP_Fragmentation': 'Non-Recon', 'DDoS-SlowLoris': 'Non-Recon', 'DDoS-HTTP_Flood': 'Non-Recon',
                      'DoS-UDP_Flood': 'Non-Recon', 'DoS-SYN_Flood': 'Non-Recon', 'DoS-TCP_Flood': 'Non-Recon', 'DoS-HTTP_Flood': 'Non-Recon', 
                      'Mirai-greeth_flood': 'Non-Recon', 'Mirai-greip_flood': 'Non-Recon', 'Mirai-udpplain': 'Non-Recon', 
                      'Recon-PingSweep': 'Recon', 'Recon-OSScan': 'Recon', 'Recon-PortScan': 'Recon', 'VulnerabilityScan': 'Recon', 'Recon-HostDiscovery': 'Recon', 
                      'DNS_Spoofing': 'Non-Recon', 'MITM-ArpSpoofing': 'Non-Recon', 
                      'BenignTraffic': 'Non-Recon', 
                      'BrowserHijacking': 'Non-Recon', 'Backdoor_Malware': 'Non-Recon', 'XSS': 'Non-Recon', 'Uploading_Attack': 'Non-Recon', 'SqlInjection': 'Non-Recon', 'CommandInjection': 'Non-Recon', 
                      'DictionaryBruteForce': 'Non-Recon'}

# Spoofing :
dict_2classes_Spoofing = {'DDoS-RSTFINFlood': 'Non-Spoofing', 'DDoS-PSHACK_Flood': 'Non-Spoofing', 'DDoS-SYN_Flood': 'Non-Spoofing', 'DDoS-UDP_Flood': 'Non-Spoofing', 'DDoS-TCP_Flood': 'Non-Spoofing', 'DDoS-ICMP_Flood': 'Non-Spoofing', 'DDoS-SynonymousIP_Flood': 'Non-Spoofing', 'DDoS-ACK_Fragmentation': 'Non-Spoofing', 'DDoS-UDP_Fragmentation': 'Non-Spoofing', 'DDoS-ICMP_Fragmentation': 'Non-Spoofing', 'DDoS-SlowLoris': 'Non-Spoofing', 'DDoS-HTTP_Flood': 'Non-Spoofing',
                      'DoS-UDP_Flood': 'Non-Spoofing', 'DoS-SYN_Flood': 'Non-Spoofing', 'DoS-TCP_Flood': 'Non-Spoofing', 'DoS-HTTP_Flood': 'Non-Spoofing', 
                      'Mirai-greeth_flood': 'Non-Spoofing', 'Mirai-greip_flood': 'Non-Spoofing', 'Mirai-udpplain': 'Non-Spoofing', 
                      'Recon-PingSweep': 'Non-Spoofing', 'Recon-OSScan': 'Non-Spoofing', 'Recon-PortScan': 'Non-Spoofing', 'VulnerabilityScan': 'Non-Spoofing', 'Recon-HostDiscovery': 'Non-Spoofing', 
                      'DNS_Spoofing': 'Spoofing', 'MITM-ArpSpoofing': 'Spoofing', 
                      'BenignTraffic': 'Non-Spoofing', 
                      'BrowserHijacking': 'Non-Spoofing', 'Backdoor_Malware': 'Non-Spoofing', 'XSS': 'Non-Spoofing', 'Uploading_Attack': 'Non-Spoofing', 'SqlInjection': 'Non-Spoofing', 'CommandInjection': 'Non-Spoofing', 
                      'DictionaryBruteForce': 'Non-Spoofing'}

# Benign :
dict_2classes_Benign = {'DDoS-RSTFINFlood': 'Non-Benign', 'DDoS-PSHACK_Flood': 'Non-Benign', 'DDoS-SYN_Flood': 'Non-Benign', 'DDoS-UDP_Flood': 'Non-Benign', 'DDoS-TCP_Flood': 'Non-Benign', 'DDoS-ICMP_Flood': 'Non-Benign', 'DDoS-SynonymousIP_Flood': 'Non-Benign', 'DDoS-ACK_Fragmentation': 'Non-Benign', 'DDoS-UDP_Fragmentation': 'Non-Benign', 'DDoS-ICMP_Fragmentation': 'Non-Benign', 'DDoS-SlowLoris': 'Non-Benign', 'DDoS-HTTP_Flood': 'Non-Benign',
                      'DoS-UDP_Flood': 'Non-Benign', 'DoS-SYN_Flood': 'Non-Benign', 'DoS-TCP_Flood': 'Non-Benign', 'DoS-HTTP_Flood': 'Non-Benign', 
                      'Mirai-greeth_flood': 'Non-Benign', 'Mirai-greip_flood': 'Non-Benign', 'Mirai-udpplain': 'Non-Benign', 
                      'Recon-PingSweep': 'Non-Benign', 'Recon-OSScan': 'Non-Benign', 'Recon-PortScan': 'Non-Benign', 'VulnerabilityScan': 'Non-Benign', 'Recon-HostDiscovery': 'Non-Benign', 
                      'DNS_Spoofing': 'Non-Benign', 'MITM-ArpSpoofing': 'Non-Benign', 
                      'BenignTraffic': 'Benign', 
                      'BrowserHijacking': 'Non-Benign', 'Backdoor_Malware': 'Non-Benign', 'XSS': 'Non-Benign', 'Uploading_Attack': 'Non-Benign', 'SqlInjection': 'Non-Benign', 'CommandInjection': 'Non-Benign', 
                      'DictionaryBruteForce': 'Non-Benign'}

# Web :
dict_2classes_Web = {'DDoS-RSTFINFlood': 'Non-Web', 'DDoS-PSHACK_Flood': 'Non-Web', 'DDoS-SYN_Flood': 'Non-Web', 'DDoS-UDP_Flood': 'Non-Web', 'DDoS-TCP_Flood': 'Non-Web', 'DDoS-ICMP_Flood': 'Non-Web', 'DDoS-SynonymousIP_Flood': 'Non-Web', 'DDoS-ACK_Fragmentation': 'Non-Web', 'DDoS-UDP_Fragmentation': 'Non-Web', 'DDoS-ICMP_Fragmentation': 'Non-Web', 'DDoS-SlowLoris': 'Non-Web', 'DDoS-HTTP_Flood': 'Non-Web',
                      'DoS-UDP_Flood': 'Non-Web', 'DoS-SYN_Flood': 'Non-Web', 'DoS-TCP_Flood': 'Non-Web', 'DoS-HTTP_Flood': 'Non-Web', 
                      'Mirai-greeth_flood': 'Non-Web', 'Mirai-greip_flood': 'Non-Web', 'Mirai-udpplain': 'Non-Web', 
                      'Recon-PingSweep': 'Non-Web', 'Recon-OSScan': 'Non-Web', 'Recon-PortScan': 'Non-Web', 'VulnerabilityScan': 'Non-Web', 'Recon-HostDiscovery': 'Non-Web', 
                      'DNS_Spoofing': 'Non-Web', 'MITM-ArpSpoofing': 'Non-Web', 
                      'BenignTraffic': 'Non-Web', 
                      'BrowserHijacking': 'Web', 'Backdoor_Malware': 'Web', 'XSS': 'Web', 'Uploading_Attack': 'Web', 'SqlInjection': 'Web', 'CommandInjection': 'Web', 
                      'DictionaryBruteForce': 'Non-Web'}

# BruteForce :
dict_2classes_BruteForce = {'DDoS-RSTFINFlood': 'Non-BruteForce', 'DDoS-PSHACK_Flood': 'Non-BruteForce', 'DDoS-SYN_Flood': 'Non-BruteForce', 'DDoS-UDP_Flood': 'Non-BruteForce', 'DDoS-TCP_Flood': 'Non-BruteForce', 'DDoS-ICMP_Flood': 'Non-BruteForce', 'DDoS-SynonymousIP_Flood': 'Non-BruteForce', 'DDoS-ACK_Fragmentation': 'Non-BruteForce', 'DDoS-UDP_Fragmentation': 'Non-BruteForce', 'DDoS-ICMP_Fragmentation': 'Non-BruteForce', 'DDoS-SlowLoris': 'Non-BruteForce', 'DDoS-HTTP_Flood': 'Non-BruteForce',
                      'DoS-UDP_Flood': 'Non-BruteForce', 'DoS-SYN_Flood': 'Non-BruteForce', 'DoS-TCP_Flood': 'Non-BruteForce', 'DoS-HTTP_Flood': 'Non-BruteForce', 
                      'Mirai-greeth_flood': 'Non-BruteForce', 'Mirai-greip_flood': 'Non-BruteForce', 'Mirai-udpplain': 'Non-BruteForce', 
                      'Recon-PingSweep': 'Non-BruteForce', 'Recon-OSScan': 'Non-BruteForce', 'Recon-PortScan': 'Non-BruteForce', 'VulnerabilityScan': 'Non-BruteForce', 'Recon-HostDiscovery': 'Non-BruteForce', 
                      'DNS_Spoofing': 'Non-BruteForce', 'MITM-ArpSpoofing': 'Non-BruteForce', 
                      'BenignTraffic': 'Non-BruteForce', 
                      'BrowserHijacking': 'Non-BruteForce', 'Backdoor_Malware': 'Non-BruteForce', 'XSS': 'Non-BruteForce', 'Uploading_Attack': 'Non-BruteForce', 'SqlInjection': 'Non-BruteForce', 'CommandInjection': 'Non-BruteForce', 
                      'DictionaryBruteForce': 'BruteForce'}

dict_2classes = [{"dict_2classes": dict_2classes_DDoS, "type": "DDoS"}, {"dict_2classes": dict_2classes_DoS, "type": "DoS"}, {"dict_2classes": dict_2classes_Mirai, "type": "Mirai"}, {"dict_2classes": dict_2classes_Recon, "type": "Recon"}, {"dict_2classes": dict_2classes_Spoofing, "type": "Spoofing"}, {"dict_2classes": dict_2classes_Benign, "type": "Benign"}, {"dict_2classes": dict_2classes_Web, "type": "Web"}, {"dict_2classes": dict_2classes_BruteForce, "type": "BruteForce"}]
print(dict_2classes[0])

{'dict_2classes': {'DDoS-RSTFINFlood': 'DDoS', 'DDoS-PSHACK_Flood': 'DDoS', 'DDoS-SYN_Flood': 'DDoS', 'DDoS-UDP_Flood': 'DDoS', 'DDoS-TCP_Flood': 'DDoS', 'DDoS-ICMP_Flood': 'DDoS', 'DDoS-SynonymousIP_Flood': 'DDoS', 'DDoS-ACK_Fragmentation': 'DDoS', 'DDoS-UDP_Fragmentation': 'DDoS', 'DDoS-ICMP_Fragmentation': 'DDoS', 'DDoS-SlowLoris': 'DDoS', 'DDoS-HTTP_Flood': 'DDoS', 'DoS-UDP_Flood': 'Non-DDoS', 'DoS-SYN_Flood': 'Non-DDoS', 'DoS-TCP_Flood': 'Non-DDoS', 'DoS-HTTP_Flood': 'Non-DDoS', 'Mirai-greeth_flood': 'Non-DDoS', 'Mirai-greip_flood': 'Non-DDoS', 'Mirai-udpplain': 'Non-DDoS', 'Recon-PingSweep': 'Non-DDoS', 'Recon-OSScan': 'Non-DDoS', 'Recon-PortScan': 'Non-DDoS', 'VulnerabilityScan': 'Non-DDoS', 'Recon-HostDiscovery': 'Non-DDoS', 'DNS_Spoofing': 'Non-DDoS', 'MITM-ArpSpoofing': 'Non-DDoS', 'BenignTraffic': 'Non-DDoS', 'BrowserHijacking': 'Non-DDoS', 'Backdoor_Malware': 'Non-DDoS', 'XSS': 'Non-DDoS', 'Uploading_Attack': 'Non-DDoS', 'SqlInjection': 'Non-DDoS', 'CommandInjection': 'Non-

#### Specific models classification

In [9]:
# Chargement des modèles spécifiques par catégorie
def load_models_by_category():
    models_by_category = {}
    for category in dict_7classes.values():
        # Chargez le modèle spécifique à la catégorie depuis le fichier local
        model_filename = f".\Models\model_{ML_neam}_{category}.joblib"
        model = joblib.load(model_filename)
        models_by_category[category] = model
    return models_by_category

# Chargez les modèles spécifiques par catégorie
models_by_category = load_models_by_category()

# Fonction pour prédire une ligne avec le modèle spécifique à la catégorie
def predict_with_category_model(row, category, models_by_category):
    if category in models_by_category:
        model = models_by_category[category]
        # Prédisez la ligne avec le modèle spécifique à la catégorie
        prediction = model.predict([row])  # Assurez-vous que "row" est un tableau 2D avec les caractéristiques attendues
        return prediction[0]  # Retournez la prédiction (assumant que le modèle renvoie un tableau de prédictions)
    else:
        return None

# Créez un dictionnaire pour stocker les prédictions classées par catégorie
classified_predictions = {category: [] for category in dict_7classes.values()}
model_name = ML_neam

# Itérez sur les prédictions et les lignes correspondantes
for index, prediction in tqdm(enumerate(y_pred)):
    # Obtenez la catégorie correspondant à la prédiction
    category = dict_7classes.get(prediction, None)

    if category:
        # Obtenez le vrai label correspondant à l'index actuel depuis y_test
        true_label = y_test[index]

        # Prédisez la ligne avec le modèle spécifique à la catégorie
        row = d_test[index]  # Assurez-vous que d_test contient les données de test dans le même ordre que y_pred
        category_prediction = predict_with_category_model(row, category, models_by_category)

        # Ajoutez la prédiction du modèle général en plus de la classification par catégorie
        general_prediction = prediction

        # Ajoutez toutes les informations au dictionnaire
        classified_predictions[category].append({
            'model_name': model_name,
            'general_prediction': general_prediction,  # Ajoutez la prédiction du modèle général
            'category_prediction': category_prediction,  # Ajoutez la prédiction spécifique à la catégorie
            'true_label': true_label,
            'row_index': index
        })

# Print classified predictions for a specific category (e.g., 'Mirai')        
# print(classified_predictions['Mirai'][:5])

# Créez une liste pour stocker les dictionnaires où category_prediction est "Non-..."
misclassified_predictions = []

# Itérez sur chaque catégorie
for category, predictions in tqdm(classified_predictions.items()):
    # Itérez sur chaque prédiction pour cette catégorie
    for prediction_info in predictions:
        # Vérifiez si category_prediction est "Non-..."
        if prediction_info['category_prediction'] == "Non-...":
            # Ajoutez ce dictionnaire à la liste des prédictions mal classées
            misclassified_predictions.append(prediction_info)

# Print the first 5 misclassified predictions
print("Misclassified predictions:")
print(misclassified_predictions[:5])

# Itérez sur les catégories
for category, predictions in classified_predictions.items():
    print(f"Catégorie : {category}")
    
    # Séparez les prédictions du modèle général et spécifique à la catégorie
    general_predictions = [item['general_prediction'] for item in predictions]
    category_predictions = [item['category_prediction'] for item in predictions]
    # print(f"general_predictions : {general_predictions[:5]}")
    # print(f"general_predictions : {len(general_predictions)}")
    # print(f"category_predictions : {category_predictions[:5]}")
    # print(f"category_predictions : {len(category_predictions)}")
    
    # Obtenez les vraies étiquettes correspondantes
    true_labels = [item['true_label'] for item in predictions]
    # print(f"true_labels : {true_labels[:5]}")
    # print(f"true_labels : {len(true_labels)}")

    true_labels_category_predictions = []
    indices = [item for index, item in enumerate(dict_2classes) if item["type"] == category]
    print(f"indices : {indices}")
    for elt in true_labels:
        true_labels_category_predictions.append(indices[0]["dict_2classes"][elt])
    # print(f"true_labels_category_predictions : {true_labels_category_predictions[:5]}")

    
    # Calculez et ajoutez l'accuracy et la précision du modèle spécifique
    print(f"Accuracy du modèle général : {accuracy_score(true_labels, general_predictions)}")
    print(f"Accuracy du modèle spécifique : {accuracy_score(true_labels_category_predictions, category_predictions)}")
    print(f"Recall du modèle général : {recall_score(true_labels, general_predictions, average='macro')}")
    print(f"Recall du modèle spécifique : {recall_score(true_labels_category_predictions, category_predictions, average='macro')}")
    print(f"Precision du modèle général : {precision_score(true_labels, general_predictions, average='macro')}")
    print(f"Precision du modèle spécifique : {precision_score(true_labels_category_predictions, category_predictions, average='macro')}")
    print(f"F1 du modèle général : {f1_score(true_labels, general_predictions, average='macro')}")
    print(f"F1 du modèle spécifique : {f1_score(true_labels_category_predictions, category_predictions, average='macro')}")
    print()


10340161it [08:09, 21121.42it/s]
100%|██████████| 8/8 [00:01<00:00,  5.81it/s]


Misclassified predictions:
[]
Catégorie : DDoS
indices : [{'dict_2classes': {'DDoS-RSTFINFlood': 'DDoS', 'DDoS-PSHACK_Flood': 'DDoS', 'DDoS-SYN_Flood': 'DDoS', 'DDoS-UDP_Flood': 'DDoS', 'DDoS-TCP_Flood': 'DDoS', 'DDoS-ICMP_Flood': 'DDoS', 'DDoS-SynonymousIP_Flood': 'DDoS', 'DDoS-ACK_Fragmentation': 'DDoS', 'DDoS-UDP_Fragmentation': 'DDoS', 'DDoS-ICMP_Fragmentation': 'DDoS', 'DDoS-SlowLoris': 'DDoS', 'DDoS-HTTP_Flood': 'DDoS', 'DoS-UDP_Flood': 'Non-DDoS', 'DoS-SYN_Flood': 'Non-DDoS', 'DoS-TCP_Flood': 'Non-DDoS', 'DoS-HTTP_Flood': 'Non-DDoS', 'Mirai-greeth_flood': 'Non-DDoS', 'Mirai-greip_flood': 'Non-DDoS', 'Mirai-udpplain': 'Non-DDoS', 'Recon-PingSweep': 'Non-DDoS', 'Recon-OSScan': 'Non-DDoS', 'Recon-PortScan': 'Non-DDoS', 'VulnerabilityScan': 'Non-DDoS', 'Recon-HostDiscovery': 'Non-DDoS', 'DNS_Spoofing': 'Non-DDoS', 'MITM-ArpSpoofing': 'Non-DDoS', 'BenignTraffic': 'Non-DDoS', 'BrowserHijacking': 'Non-DDoS', 'Backdoor_Malware': 'Non-DDoS', 'XSS': 'Non-DDoS', 'Uploading_Attack': 'Non-DD

#### Example of result for specific models

##### Catégorie : DDoS
- Accuracy du modèle général : 0.9995087462958314
- Accuracy du modèle spécifique : 0.99975311079731
- Recall du modèle général : 0.3620989296273269
- Recall du modèle spécifique : 0.8811709448614627
- Precision du modèle général : 0.36092101193637194
- Precision du modèle spécifique : 0.7073978915991486
- F1 du modèle général : 0.3615065795760696
- F1 du modèle spécifique : 0.7686135600309036

##### Catégorie : DoS
- Accuracy du modèle général : 0.9994045974894884
- Accuracy du modèle spécifique : 0.9997232243250105
- Recall du modèle général : 0.12115331796830618
- Recall du modèle spécifique : 0.8878634870657732
- Precision du modèle général : 0.12079666355870726
- Precision du modèle spécifique : 0.8693157354027607
- F1 du modèle général : 0.12097416497624888
- F1 du modèle spécifique : 0.8783621392243404

##### Catégorie : Mirai
- Accuracy du modèle général : 0.9982951235392893
- Accuracy du modèle spécifique : 0.9998269421884103
- Recall du modèle général : 0.11530180471348712
- Recall du modèle spécifique : 0.9714934090995118
- Precision du modèle général : 0.11518757271179653
- Precision du modèle spécifique : 0.9446871455104033
- F1 du modèle général : 0.11524466015412763
- F1 du modèle spécifique : 0.9576972769823738

##### Catégorie : Recon
- Accuracy du modèle général : 0.7124337367954922
- Accuracy du modèle spécifique : 0.8897303264064486
- Recall du modèle général : 0.12499873197954459
- Recall du modèle spécifique : 0.8367776825796007
- Precision du modèle général : 0.09960998288850838
- Precision du modèle spécifique : 0.8214734381516187
- F1 du modèle général : 0.11038236349127492
- F1 du modèle spécifique : 0.8287451950185816

##### Catégorie : Spoofing
- Accuracy du modèle général : 0.7447941932944633
- Accuracy du modèle spécifique : 0.8886254839684036
- Recall du modèle général : 0.05486950617309349
- Recall du modèle spécifique : 0.809086848249182
- Precision du modèle général : 0.04416654236416736
- Precision du modèle spécifique : 0.8158432374149169
- F1 du modèle général : 0.04889291703232908
- F1 du modèle spécifique : 0.8123938376362991

##### Catégorie : Benign
- Accuracy du modèle général : 0.9094352375810381
- Accuracy du modèle spécifique : 0.8995973342455478
- Recall du modèle général : 0.030303030303030304
- Recall du modèle spécifique : 0.686914162926673
- Precision du modèle général : 0.027558643563061758
- Precision du modèle spécifique : 0.6933828521562496
- F1 du modèle général : 0.02886575362249451
- F1 du modèle spécifique : 0.6900695756404035

##### Catégorie : Web
- Accuracy du modèle général : 0.3873473917869034
- Accuracy du modèle spécifique : 0.8671317583637228
- Recall du modèle général : 0.18364327958179189
- Recall du modèle spécifique : 0.8590589688651405
- Precision du modèle général : 0.09039062167657357
- Precision du modèle spécifique : 0.8744895183794599
- F1 du modèle général : 0.11920474718333268
- F1 du modèle spécifique : 0.8633069861434141

##### Catégorie : BruteForce
- Accuracy du modèle général : 0.5214207250091542
- Accuracy du modèle spécifique : 0.9549615525448554
- Recall du modèle général : 0.0625
- Recall du modèle spécifique : 0.9542029521246873
- Precision du modèle général : 0.032588795313072134
- Precision du modèle spécifique : 0.9558881899683329
- F1 du modèle général : 0.042839951865222624
- F1 du modèle spécifique : 0.9548064306117202

#### Some tests

In [7]:
gc.collect()

2203